In [2]:
!pip install langchain langchain-community langchain-neo4j langchain_openai pypdf

  Using cached langchain_neo4j-0.3.0-py3-none-any.whl.metadata (4.5 kB)
Using cached langchain_neo4j-0.3.0-py3-none-any.whl (38 kB)


In [3]:
import os
os.environ["OPENAI_API_KEY"] = "sk" #openai 키 입력

In [4]:
from langchain.docstore.document import Document # 랭체인의 문서 객체
from langchain.text_splitter import CharacterTextSplitter # 문서를 작은 청크(Chunk)로 분리
from langchain.document_loaders import PyPDFLoader  # PDF 파일을 로드하고 텍스트를 추출
from langchain_openai import OpenAIEmbeddings # OpenAI의 임베딩 모델을 사용하여 텍스트를 벡터로 변환
from langchain.vectorstores import Neo4jVector # Neo4j 벡터 데이터베이스를 사용하여 검색

# PDF 파일 로드하고 텍스트를 추출
loader = PyPDFLoader("d:/data/차세대 한국형 스마트팜 개발.pdf")

# PDF에서 문서를 로드
documents = loader.load() # PDF 파일에서 텍스트를 추출하여 documents에 저장

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0) # 텍스트를 1000자씩 분할, 중복 없음
docs = text_splitter.split_documents(documents) # PDF에서 추출한 문서를 청크(Chunk)로 변환하여 docs에 저장

# OpenAI의 임베딩 모델을 사용하여 텍스트를 벡터로 변환
embeddings = OpenAIEmbeddings()

# Neo4j와 연결
url = "neo4j://localhost:7687"  # Neo4j URI
username = "neo4j"  # Neo4j 사용자명
password = "Young1457!"  # Neo4j 비밀번호 입력

db = Neo4jVector.from_documents(
    docs, embeddings, url=url, username=username, password=password, database="neo4j"
) # Neo4j에 벡터 데이터를 저장하고 검색 가능하도록 설정

query = "스마트팜을 위한 ICT 기술은?" # 사용자 질문
docs_with_score = db.similarity_search_with_score(query, k=5) # 질문과 가장 유사한 문서 5개(k=5)를 검색

# 검색된 문서들과 유사도 점수를 출력, score 값이 낮을수록 더 유사한 문서
for doc, score in docs_with_score:
    print("-" * 80)
    print("Score: ", score)
    print(doc.page_content)
    print("-" * 80)

--------------------------------------------------------------------------------
Score:  0.9458465576171875
제어기 등 ICT 기기 산업을 활성화 시키는데 필수적이다.
ICT 핵심부품 및 기기 표준화
 스마트온실 단체표준 제정
파이팅! 한국형 스마트팜
사회 전반에서 4차 산업 혁명에 대한 많은 관심을 가지고 사물인터넷, 빅데이터, 인공지능, 로봇 등을 활용하는 분야를
확장하고 있다. 농업분야에서도 스마트 기기를 농사에 효과적으로 접목할 수 있는 ICT분야 연구개발이 필요하다.
첨단화되는 스마트 기기와 다양한 실시간 정보를 활용해 편리성과 생산성을 높이는 스마트팜 연구개발은 국내농가의
소득증대뿐만 아니라 농업 생산기술과 시스템 수출의 길을 열고, 우리 농업기술의 국제경쟁력을 높이는 데에도 크게 기여할
것으로 기대 된다. 가까운 장래에 한국형 스마트 팜의 기반이 구축되고 우리 경제의 미래 성장 동력으로 자리매김 할 우리
농업의 모습을 기대해 본다.
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Score:  0.928070068359375
빅데이터를 활용한 생산성 향상
3세대 - 에너지 최적화 및 로봇 자동화 등 스마트팜 통합시스템 수출
한국형 스마트팜을 기술적으로 완성하는 3세대 스마트팜 모델은 1세대 편의성 향상과 2세대 생산성 향상 기술의 토대 위에 온실 및
축사에너지 시스템의 최적화와 다양한 로봇을 활용한 무인화․자동화시스템을 구현해 스마트팜 전 과정의 통합제어 및 생산 관리를 가능하게
하는 수준이다.
세대별 한국형 스마트팜 기술 구성
3세대 모델은 국내 스마트팜 농가의 규모화와 생력화를 통한 생산비 절감으로 경쟁력을 향상할 뿐만 아니

In [8]:
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQAWithSourcesChain

# 사용자가 질문하면 Neo4j에서 관련 문서를 검색할 수 있도록 설정
retriever = db.as_retriever() 

# Retrieval 기반 질의응답 체인 생성
chain = RetrievalQAWithSourcesChain.from_chain_type(
    ChatOpenAI(temperature=0),
    chain_type="stuff", # 검색된 문서를 하나로 결합하여 GPT에게 전달
    retriever=retriever # Neo4j에서 검색된 문서를 기반으로 답변 생성
)

query = "스마트팜이란?"
response = chain.invoke({"question": query}) # LLM 모델의 응답 반환
print(response)

{'question': '스마트팜이란?', 'answer': '스마트팜은 사물인터넷, 빅데이터, 인공지능, 로봇 등을 활용하여 농산물의 생육환경을 최적상태로 관리하고 노동력 절감과 생산성 향상을 구현하는 효율적인 농업형태이다. 차세대 한국형 스마트팜은 4기관 19개 전담부서가 협업하여 핵심 요소 및 원천 기반기술의 확보를 위해 연구 역량을 집중하고 있으며, 국내 농업여건에 적합하게 기술수준별로 스마트팜 모델을 3가지 단계로 구분하여 개발을 추진하고 있다. 1세대 스마트팜은 원격 모니터링과 제어로 편의성을 향상시키는 것이 목적이다. 2세대 스마트팜은 지능형 정밀생육관리로 생산성을 향상시키는 것이 목표이며, 3세대는 에너지 최적화 및 로봇 자동화 등 스마트팜 통합시스템을 수출하는 것을 목표로 한다.\n', 'sources': 'd:/data/차세대 한국형 스마트팜 개발.pdf'}


In [9]:
query = "스마트팜을 위한 ICT 기술은?"
response = chain.invoke({"question": query}, return_only_outputs=True)
print(response)

{'answer': "I don't know.\n", 'sources': ''}


In [11]:
query = "스마트팜을 위한 ICT 기기는 어떤 것들이 있어?"
response = chain.invoke({"question": query}, return_only_outputs=True)
print(response)

{'answer': 'ICT 기기로는 제어기, 사물인터넷, 빅데이터, 인공지능, 로봇 등이 사용된다.\n', 'sources': 'd:/data/차세대 한국형 스마트팜 개발.pdf'}
